# Le notebook que yohan n'a pas fait

In [9]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
years = ['19', '20', '21', '22']
files_name = ["caracteristiques-20", "lieux-20", "usagers-20", "vehicules-20"]

for file_name in files_name:
    result = []
    print(f"\n///////////////{file_name}///////////////")
    for year in years:
        file_path = f'./Dataset/{year}/{file_name}{year}.csv'
        sep = ';'
        df = pd.read_csv(file_path, sep=sep, low_memory=False)
        # if 'id_usager' in df.columns:
        #     df = df.drop(columns=['id_usager'])
        result.append(df)
        print(df.columns.tolist())
    combined_df = pd.concat(result, ignore_index=True)
    combined_df.to_csv(f'./Dataset/all_years/{file_name}{years[0]}-{years[-1]}.csv', index=False)

In [10]:
data = pd.read_csv('accidents-19-22.csv', low_memory=False)

pd.DataFrame(data)

,Accident_Id,jour,mois,an,hrmn,lum,dep,com,agg,int,...,actp,etatp,senc,catv,obs,obsm,choc,manv,motor,occutc
0,201900000001,30,11,2019,01:30,4,93,93053,1,1,...,-1,-1,2,7,0,2,5,23,1,NaN
1,201900000001,30,11,2019,01:30,4,93,93053,1,1,...,-1,-1,2,7,0,2,5,23,1,NaN
2,201900000001,30,11,2019,01:30,4,93,93053,1,1,...,-1,-1,2,17,1,0,3,11,1,NaN
3,201900000002,30,11,2019,02:50,3,93,93066,1,1,...,-1,-1,1,7,4,0,1,0,1,NaN
4,201900000003,28,11,2019,15:15,1,92,92036,1,1,...,0,-1,1,7,0,2,1,2,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494177,202200055301,1,1,2022,08:40,1,81,81099,1,3,...,0,-1,2,7,0,0,8,19,1,NaN
494178,202200055301,1,1,2022,08:40,1,81,81099,1,3,...,0,-1,2,7,0,0,8,19,1,NaN
494179,202200055301,1,1,2022,08:40,1,81,81099,1,3,...,0,-1,2,7,0,2,1,1,1,NaN
494180,202200055302,1,3,2022,16:55,1,41,41018,2,1,...,-1,-1,1,33,0,2,1,1,1,NaN


In [12]:
df = pd.DataFrame(data.groupby(['Accident_Id']))

df[0]

0         (201900000001,)
1         (201900000002,)
2         (201900000003,)
3         (201900000004,)
4         (201900000005,)
               ...       
218399    (202200055298,)
218400    (202200055299,)
218401    (202200055300,)
218402    (202200055301,)
218403    (202200055302,)
Name: 0, Length: 218404, dtype: object